### Using dataset from https://github.com/tomasonjo/blog-datasets/blob/main/movies/movies_small.csv

In [28]:
NEO4J_URI="neo4j+s://1f7a937f.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="aYCioexIrrl7g34zoj0XWnGTECT0a1NZkCrGDjXBI_c"

In [29]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [30]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [31]:
movie_query = """

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE (m:Movie {id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director IN split(row.director, '|') | 
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor IN split(row.actors, '|') | 
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre IN split(row.genres, '|') | 
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [32]:
#WITH row WHERE row.movieId IS NOT NULL AND trim(row.movieId) <> ''

In [33]:
movie_query

"\n\nLOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row\n\nMERGE (m:Movie {id: row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director IN split(row.director, '|') | \n    MERGE (p:Person {name: trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor IN split(row.actors, '|') | \n    MERGE (p:Person {name: trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre IN split(row.genres, '|') | \n    MERGE (g:Genre {name: trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n"

In [34]:
graph.query(movie_query)

[]

In [35]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, POB: STRING, YOB: INTEGER}
Company {name: STRING}
entreprenur {name: STRING, POB: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, born: INTEGER}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [36]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [37]:
from langchain_groq import ChatGroq
llm = ChatGroq(api_key = groq_api_key, model = "llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1141f6330>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1141f6f60>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [38]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph = graph, llm=llm, verbose = True, allow_dangerous_requests = True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x1141ee6c0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1141f6330>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1141f6f60>, model_name='llama-3.1-8b-instant

In [39]:
response = chain.invoke({"query": "who directed Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(c:Person) WHERE m.title = 'Casino' RETURN p.name AS Director, c.name AS Actor
Full Context:
[{'Director': 'Martin Scorsese', 'Actor': 'Robert De Niro'}, {'Director': 'Martin Scorsese', 'Actor': 'Joe Pesci'}, {'Director': 'Martin Scorsese', 'Actor': 'Sharon Stone'}, {'Director': 'Martin Scorsese', 'Actor': 'James Woods'}]

> Finished chain.


{'query': 'who directed Casino', 'result': 'Martin Scorsese directed Casino.'}

In [42]:
response = chain.invoke({"query": "who were the actors in Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = "Casino" RETURN p.name
Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


{'query': 'who were the actors in Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, James Woods were the actors in Casino.'}